In [ ]:
import requests
import crdncls as cr
import psycopg2
import logging

logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
handler = logging.FileHandler('pruebaRetanqueo203.log')
handler.setLevel(logging.DEBUG)
handler.setFormatter(logging.Formatter('%(asctime)s|%(levelname)s|%(message)s'))
logging.getLogger('').addHandler(handler)

logging.info("Esta es la prueba para la oficina 203 poder verificar que si un credito fondeado por cualquier fondeador y si tiene al menos un pago se pueda retanquear")

cr.token
heads = open("./noms.txt", "r")
head = {'Authorization': 'Bearer {}'.format(heads.read())}

conexcion = psycopg2.connect(user=cr.user, 
                             password=cr.password, 
                             host=cr.host, 
                             port=cr.port, database=cr.database) 
cursor = conexcion.cursor()
cursor.execute(r"select count(cf.id_credito), cl.identificacion FROM(SELECT * FROM obtener_creditos_fondeados(current_date)) cf INNER join cliente cl ON cl.id = cf.id_cliente where cf.id_fondeador  not IN (0) group by cl.identificacion limit 500;")
data = cursor.fetchall()
cursor.close()
conexcion.close()

for i in data:
    logging.info(f"Para el cliente {i[1]} verificamos si tiene retanqueos:")
    body = {"tipoIdentificacion": "CC","identificacion": i[1]}
    url = (f'https://development.excelcredit.co/instancia2/api/excelcredit/onboardingservices/clientes/creditos-activos/203')
    response = requests.post(url, headers=head, json=body)
    logging.debug(response.text)
    if len(response.json()) != 0:
        logging.info(f"El cliente {i[1]} si tiene {len(response.json())} crédito activos para la oficina 203")
        logging.info(f"Los créditos activos son:")
        for j in response.json():
            idCredito = j['id']
            conexcion = psycopg2.connect(user=cr.user, 
                             password=cr.password, 
                             host=cr.host, 
                             port=cr.port, database=cr.database) 
            cursor = conexcion.cursor()
            cursor.execute(f"select cf.id_credito, cf.id_fondeador, cf.estado_credito, cl.identificacion, cf.id_cliente, cf.es_vendido, public.verificar_incorporacion_primera_cuota(cf.id_credito) as tiene_incorporacion FROM(SELECT * FROM obtener_creditos_fondeados(current_date)) cf INNER join cliente cl ON cl.id = cf.id_cliente where cf.id_fondeador  not IN (0) and cf.id_credito  = {idCredito};")
            data = cursor.fetchall()
            # logging.info(data)
            logging.info(f"idCredito: {data[0][0]}, fondeador: {data[0][1]}, estadoCredito: {data[0][2]}, identificacion: {data[0][3]}, esVendido: {data[0][5]}, tieneIncorporacion: {data[0][6]}")
            cursor.close()
            conexcion.close()
    else:
        logging.info(f"El cliente {i[1]} no tiene créditos activos para la oficina 203")

        

In [3]:
import requests
import crdncls as cr


cr.token
heads = open("./noms.txt", "r")
head = {'Authorization': 'Bearer {}'.format(heads.read())}

data = {"idPagaduria":401,"idLineaCredito":1,"idOficina":5,"actividadCliente":"PENSIONADO","tasa":2.09,"plazo":72,"monto":17000000,"diasInteresesIniciales":120,"ingresos":3000000,"cliente":{"identificacion":"29770779","fechaNacimiento":"1964-06-08"},"descuentosLey":10000,"descuentosNomina":10000,"carteras":[],"departamento":"CO>Antioquia","genero":"F","nivelEducativo":"TECNICO","tipoContrato":"Asignación de Retiro","productPricing":{"rate":2.09,"suretyBond":7.87,"creditStudy":4.32,"productGroupId":4,"deterioration":0.010698051080747086,"rates":[{"rate":2.09,"suretyBond":7.87,"creditStudy":4.32},{"rate":2.14,"suretyBond":8.77,"creditStudy":3.42},{"rate":2.19,"suretyBond":9.67,"creditStudy":2.52},{"rate":2.24,"suretyBond":10.57,"creditStudy":1.62},{"rate":2.29,"suretyBond":11.43,"creditStudy":0.76},{"rate":2.37,"suretyBond":12.19,"creditStudy":0}],"groupNumber":4},"totalActivos":"30000000","estrato":"Estrato 3","simulador1":True}

url = (f'https://development.excelcredit.co/instancia2/api/excelcredit/onboardingservices/simulacion/simular')
response = requests.post(url, headers=head, json=data)

print(response.text)

Jwt is expired
